In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.signal
import sympy as sym
import numpy as np
import scipy as sp

from sympy import sin, cos

sns.set()
plt.rcParams["figure.figsize"] = (8,8)

# Part A


Consider a FIR (finite impulse response) discrete time filter such that

$$
    x_n \sum\limits_{m=0}^{N_h-1} h_m g_{n-m}
$$

$x_n$ is the filter output, $h_m$ are the FIR filter coefficients and, $g_n$ is the input excitation sequence which is assumed to be known. It is convenient if it is a white noise sequence of normal random samples of unit variance and zero mean. A common problem is that $x_n$ is corrupted by additive noise such that the actual observation is


$$
    x_n \sum\limits_{m=0}^{N_h-1} h_m g_{n-m} + w_n
$$

where

$$
    w \sim \mathcal{N}(0, \sigma^2)
$$

Write a routine that estimates the values of $h_m$ given an input excitation sequence of $g_n$ that is assumed to be known and with
$w_n$ as iid (independent and identically distributed) zero mean Gaussian with
$\sigma^2 = 0.1$. Try out your routine with the filter coefficients of: `h = [1,2,-1,4,1]`;

In [2]:
N = 1000
h = np.array([1, 2, -1, 4, 1])
g = np.random.randn(N)
sigma = 0.1

x_conv = sp.signal.convolve(g, h)
H = np.zeros((N + len(h) - 1, len(h)))

In [3]:
for n in range(N + len(h)):
    for m in range(len(h)):
        g_idx = n - m
        if 0 <= g_idx < N:
            H[n,m] = g[g_idx]

In [4]:
x_H = H @ h.T

In [5]:
np.sum(x_H - x_conv)

1.6854573292590658e-14

In [6]:
x = H @ h.T
x += np.random.randn(*x.shape) * sigma

### The least squares solution

The least squares solution is:
$$
    \hat h = \hat \theta = (H^T H)^{-1} H^T x
$$

In [7]:
h_hat = (np.linalg.inv(H.T @ H)) @ H.T @ x

In [8]:
# All values should be close
h, h_hat

(array([ 1,  2, -1,  4,  1]),
 array([ 0.99910565,  2.00326773, -0.99679788,  4.00291456,  1.00310761]))

# Part B

What is the expected deviation of the FIR coefficients in part A based on Fisher Information assuming that N=1000 data samples are used. Is there agreement?

In [9]:
J = H.T @ H / sigma # Equivalent to the H^T Q^{-1} H since Q is diagonal
J_inv = np.linalg.inv(J)
expected_dev = np.sqrt(np.diag(J_inv))

In [10]:
expected_dev

array([0.01005894, 0.01005895, 0.01005858, 0.01005895, 0.01005894])